# Import libraries

In [1]:
import requests
import pandas as pd
import numpy as np

# Data preprocessing

In [2]:
def fetch_historical_data(symbol, interval, limit=10000):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Fetch data for different intervals
df_1m = fetch_historical_data('BTCUSDT', '1m')
df_1h = fetch_historical_data('BTCUSDT', '1h')
df_4h = fetch_historical_data('BTCUSDT', '4h')
df_1d = fetch_historical_data('BTCUSDT', '1d')

# Prepare the training data
def prepare_data(data):
    data['returns'] = data['close'].pct_change()
    data.dropna(inplace=True)
    return data

df_1m = prepare_data(df_1m)
df_1h = prepare_data(df_1h)
df_4h = prepare_data(df_4h)
df_1d = prepare_data(df_1d)

# Modeling

In [3]:
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
frequency = '1m'

In [5]:
data = df_1m['close']
train_data = data.iloc[:int(len(data)*0.8)]
test_data = data.iloc[int(len(data)*0.8):]
train_label = train_data.shift(-1)
train_label.iloc[-1] = test_data.iloc[-1]

train_pool = Pool(train_data, train_label)
test_pool = Pool(test_data)

/opt/anaconda3/envs/CSS483/lib/python3.12/site-packages/catboost/core.py:941: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(data[0], Iterable):


In [6]:
model = CatBoostRegressor()
model.fit(train_pool)

Learning rate set to 0.039517
0:	learn: 382.8099571	total: 56ms	remaining: 55.9s
1:	learn: 369.3534854	total: 56.3ms	remaining: 28.1s
2:	learn: 356.3109125	total: 56.6ms	remaining: 18.8s
3:	learn: 343.7510775	total: 57ms	remaining: 14.2s
4:	learn: 331.9690223	total: 57.4ms	remaining: 11.4s
5:	learn: 320.5623577	total: 57.7ms	remaining: 9.56s
6:	learn: 309.4482058	total: 58.1ms	remaining: 8.24s
7:	learn: 299.0021696	total: 58.4ms	remaining: 7.25s
8:	learn: 288.9362699	total: 58.8ms	remaining: 6.47s
9:	learn: 278.9323880	total: 59.2ms	remaining: 5.86s
10:	learn: 269.2140398	total: 59.5ms	remaining: 5.35s
11:	learn: 259.8833683	total: 59.9ms	remaining: 4.93s
12:	learn: 251.0139713	total: 60.2ms	remaining: 4.57s
13:	learn: 242.5311558	total: 60.6ms	remaining: 4.27s
14:	learn: 234.4714029	total: 60.9ms	remaining: 4s
15:	learn: 226.5556318	total: 61.3ms	remaining: 3.77s
16:	learn: 219.8312577	total: 61.4ms	remaining: 3.55s
17:	learn: 212.5897961	total: 61.8ms	remaining: 3.37s
18:	learn: 205.

In [7]:
train_preds = model.predict(train_pool)
test_preds = model.predict(test_pool)

In [8]:
print(f'R2: {r2_score(train_data, train_preds):.4f}')
print(f'MSE: {mean_squared_error(train_data, train_preds):.4f}')

R2: 0.9969
MSE: 485.6485


In [9]:
print(f'R2: {r2_score(test_data, test_preds):.4f}')
print(f'MSE: {mean_squared_error(test_data, test_preds):.4f}')

R2: 0.9883
MSE: 363.2836


# Using Trading Agent

In [10]:
import sys
sys.path.append('..')

In [11]:
from ExampleAgent import ExampleAgent

In [12]:
# Backtesting function
def backtest(agent, data):
    agent.train_model(data)
    for timestamp, row in data.iterrows():
        agent.trade(data.loc[:timestamp])
    return agent.get_portfolio_value(row['close'])

# Initialize agents
agent_1m = ExampleAgent('CatBoost Agent 1m')
agent_1h = ExampleAgent('CatBoost Agent 1h')
agent_4h = ExampleAgent('CatBoost Agent 4h')
agent_1d = ExampleAgent('CatBoost Agent 1d')

# Backtest each agent
portfolio_value_1m = backtest(agent_1m, df_1m)
portfolio_value_1h = backtest(agent_1h, df_1h)
portfolio_value_4h = backtest(agent_4h, df_4h)
portfolio_value_1d = backtest(agent_1d, df_1d)
print(f"Portfolio Value for 1m Interval: {portfolio_value_1m}")
print(f"Portfolio Value for 1h Interval: {portfolio_value_1h}")
print(f"Portfolio Value for 4h Interval: {portfolio_value_4h}")
print(f"Portfolio Value for 1d Interval: {portfolio_value_1d}")

/opt/anaconda3/envs/CSS483/lib/python3.12/site-packages/catboost/core.py:941: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(data[0], Iterable):


2024-07-12 01:48:34.483299: CatBoost Agent 1m - Buy at 58065.48
2024-07-12 01:48:34.483861: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.484272: CatBoost Agent 1m - Sell at 58114.0
2024-07-12 01:48:34.484661: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.485037: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.485400: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.485763: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.486119: CatBoost Agent 1m - Buy at 57824.0
2024-07-12 01:48:34.486481: CatBoost Agent 1m - Sell at 57877.99
2024-07-12 01:48:34.486839: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.487191: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.487544: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.487895: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.488247: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.488597: CatBoost Agent 1m - Buy at 57908.01
2024-07-12 01:48:34.488949: CatBoost Agent 1m - Hold
2024-07-12 01:48:34.489295: CatBoost Agent 1m - Sell at 57974.59
2024-07-12 01:48:34.489647: CatB